# Interactive Polygon Drawing Tool

This interactive polygon drawing tool provides:

✅ **Easy polygon drawing** with intuitive map controls  
✅ **Real-time WKT export** for immediate use with phidown  
✅ **Polygon editing and deletion** capabilities  
✅ **WKT import functionality** to visualize existing polygons  
✅ **30+ premium basemap layers** including satellite imagery, topographic, dark themes, and specialized scientific maps  
✅ **Seamless integration** with phidown search functionality  

## Features

- Interactive map with drawing controls
- Real-time coordinate extraction
- WKT format output (compatible with phidown search)
- Clear and reset functionality
- Multiple polygon support
- **30+ basemap layers** organized in 6 categories:
  - 🛰️ **Satellite & Imagery** (Esri, NASA, Stadia)
  - 🗺️ **Street & Road Maps** (OSM, CartoDB, Esri)
  - ⛰️ **Topographic & Terrain** (Esri, OpenTopo, Stadia)
  - 🌑 **Dark Themes** (CartoDB, Stadia)
  - 🎨 **Artistic & Special** (Watercolor, National Geographic)
  - ❄️ **Specialized Scientific** (Arctic, Climate, NASA Earth Science)

### Imports

In [1]:
# Import required libraries
from ipyleaflet import (
    Map, 
    DrawControl, 
    GeoJSON, 
    LayersControl,
    basemaps,
    Marker,
    Popup
)
from ipywidgets import (
    VBox, 
    HBox, 
    Button, 
    Output, 
    HTML, 
    Textarea,
    Label,
    Layout
)
import json
from shapely.geometry import Polygon, mapping
from shapely import wkt
from typing import List, Dict, Any, Optional

print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


## Interactive Polygon Drawing Tool Class

Let's create a comprehensive class that handles all the polygon drawing functionality:

In [2]:
class InteractivePolygonTool:
    """
    Interactive polygon drawing tool using ipyleaflet.
    
    This class provides functionality to draw polygons on an interactive map,
    extract coordinates in WKT format, and manage multiple polygons.
    
    Attributes:
        map (Map): The ipyleaflet map widget
        draw_control (DrawControl): Drawing control for the map
        output (Output): Output widget for displaying information
        wkt_output (Textarea): Text area for WKT output
        polygons (List[Dict]): List of drawn polygons with metadata
        basemap_layers (Dict): Dictionary of available basemap layers
    """
    
    def __init__(self, center: tuple = (45.0, 0.0), zoom: int = 2, 
                 basemap=basemaps.OpenStreetMap.Mapnik, show_basemap_switcher: bool = True):
        """
        Initialize the interactive polygon tool.
        
        Args:
            center (tuple): Initial map center coordinates (lat, lon)
            zoom (int): Initial zoom level
            basemap: Initial basemap to use for the map
            show_basemap_switcher (bool): Whether to show basemap switcher controls
        """
        self.polygons: List[Dict[str, Any]] = []
        self.show_basemap_switcher = show_basemap_switcher
        self._setup_basemap_layers()
        self._setup_map(center, zoom, basemap)
        self._setup_controls()
        self._setup_ui()
    
    def _setup_basemap_layers(self) -> None:
        """
        Setup available basemap layers organized by categories.
        """
        self.basemap_layers = {
            # === SATELLITE & IMAGERY ===
            '🛰️ Esri World Imagery': basemaps.Esri.WorldImagery,
            '🌍 NASA Blue Marble': basemaps.NASAGIBS.BlueMarble,
            '🌙 NASA Earth at Night': basemaps.NASAGIBS.ViirsEarthAtNight2012,
            '🛰️ NASA MODIS True Color': basemaps.NASAGIBS.ModisTerraTrueColorCR,
            '🛰️ NASA VIIRS True Color': basemaps.NASAGIBS.ViirsTrueColorCR,
            '🛰️ Stadia Satellite': basemaps.Stadia.AlidadeSatellite,
            
            # === STREET & ROAD MAPS ===
            '🗺️ OpenStreetMap': basemaps.OpenStreetMap.Mapnik,
            '🗺️ Esri World Street': basemaps.Esri.WorldStreetMap,
            '🗺️ CartoDB Positron': basemaps.CartoDB.Positron,
            '🗺️ CartoDB Voyager': basemaps.CartoDB.Voyager,
            '🗺️ Stadia OSM Bright': basemaps.Stadia.OSMBright,
            '🗺️ Stadia Alidade Smooth': basemaps.Stadia.AlidadeSmooth,
            
            # === TOPOGRAPHIC & TERRAIN ===
            '⛰️ Esri World Topo': basemaps.Esri.WorldTopoMap,
            '⛰️ Esri World Terrain': basemaps.Esri.WorldTerrain,
            '⛰️ Esri World Physical': basemaps.Esri.WorldPhysical,
            '⛰️ Esri Shaded Relief': basemaps.Esri.WorldShadedRelief,
            '⛰️ Stadia Terrain': basemaps.Stadia.StamenTerrain,
            '⛰️ Stadia Outdoors': basemaps.Stadia.Outdoors,
            '⛰️ OpenTopoMap': basemaps.OpenTopoMap,
            
            # === DARK THEMES ===
            '🌑 CartoDB Dark Matter': basemaps.CartoDB.DarkMatter,
            '🌑 Stadia Alidade Dark': basemaps.Stadia.AlidadeSmoothDark,
            '🌑 Stadia Toner': basemaps.Stadia.StamenToner,
            
            # === ARTISTIC & SPECIAL ===
            '🎨 Stadia Watercolor': basemaps.Stadia.StamenWatercolor,
            '🗺️ Esri National Geographic': basemaps.Esri.NatGeoWorldMap,
            '🌊 Esri Ocean Basemap': basemaps.Esri.OceanBasemap,
            '🌫️ Esri World Gray Canvas': basemaps.Esri.WorldGrayCanvas,
            
            # === SPECIALIZED ===
            '❄️ Esri Arctic Imagery': basemaps.Esri.ArcticImagery,
            '🧊 NASA Ice Velocity': basemaps.NASAGIBS.MEaSUREsIceVelocity3031,
            '🌡️ NASA Land Surface Temp': basemaps.NASAGIBS.ModisTerraLSTDay,
            '☁️ NASA Snow Cover': basemaps.NASAGIBS.ModisTerraSnowCover
        }
    
    def _setup_map(self, center: tuple, zoom: int, basemap) -> None:
        """
        Setup the main map widget.
        
        Args:
            center (tuple): Map center coordinates
            zoom (int): Initial zoom level
            basemap: Basemap to use
        """
        self.map = Map(
            center=center,
            zoom=zoom,
            basemap=basemap,
            scroll_wheel_zoom=True,
            layout=Layout(height='500px', width='100%')
        )
        
        # Add layer control
        layers_control = LayersControl(position='topright')
        self.map.add_control(layers_control)
        
        # Add basemap layers as selectable layers
        if self.show_basemap_switcher:
            for name, layer in self.basemap_layers.items():
                if layer != basemap:  # Don't add the current basemap again
                    self.map.add_layer(layer)
    
    def _setup_controls(self) -> None:
        """
        Setup drawing controls for the map.
        """
        self.draw_control = DrawControl(
            polygon={
                'shapeOptions': {
                    'fillColor': '#3388ff',
                    'color': '#0000ff',
                    'fillOpacity': 0.3,
                    'weight': 2
                }
            },
            rectangle={
                'shapeOptions': {
                    'fillColor': '#ff3333',
                    'color': '#ff0000',
                    'fillOpacity': 0.3,
                    'weight': 2
                }
            },
            polyline={},
            circle={},
            circlemarker={},
            marker={},
            edit=True,
            remove=True
        )
        
        # Add event handlers using observe method
        self.draw_control.on_draw(self._handle_draw)
        # For edit and delete, we'll use a single observer on the data attribute
        self.draw_control.observe(self._handle_data_change, names=['data'])
        
        self.map.add_control(self.draw_control)
    
    def _setup_ui(self) -> None:
        """
        Setup the user interface widgets.
        """
        # Output widget for messages
        self.output = Output()
        
        # WKT output textarea
        self.wkt_output = Textarea(
            placeholder='WKT coordinates will appear here...',
            description='WKT Output:',
            layout=Layout(height='100px', width='100%'),
            style={'description_width': 'initial'}
        )
        
        # Control buttons
        self.clear_button = Button(
            description='Clear All',
            button_style='warning',
            icon='trash'
        )
        self.clear_button.on_click(self._clear_all)
        
        self.copy_button = Button(
            description='Copy WKT',
            button_style='info',
            icon='copy'
        )
        self.copy_button.on_click(self._copy_wkt)
        
        # Basemap switcher if enabled
        if self.show_basemap_switcher:
            from ipywidgets import Dropdown
            self.basemap_dropdown = Dropdown(
                options=list(self.basemap_layers.keys()),
                value='🗺️ OpenStreetMap',
                description='Basemap:',
                style={'description_width': 'initial'}
            )
            self.basemap_dropdown.observe(self._change_basemap, names='value')
        
        # Load WKT functionality
        self.wkt_input = Textarea(
            placeholder='Paste WKT string here to visualize...',
            description='Load WKT:',
            layout=Layout(height='80px', width='100%'),
            style={'description_width': 'initial'}
        )
        
        self.load_button = Button(
            description='Load WKT',
            button_style='success',
            icon='upload'
        )
        self.load_button.on_click(self._load_wkt)
    
    def _change_basemap(self, change) -> None:
        """
        Change the basemap when dropdown selection changes.
        
        Args:
            change: The change event from the dropdown
        """
        new_basemap_name = change['new']
        new_basemap = self.basemap_layers[new_basemap_name]
        
        # Remove current basemap and add new one
        # Clear all layers first, then add the new basemap
        layers_to_remove = [layer for layer in self.map.layers if hasattr(layer, 'url')]
        for layer in layers_to_remove:
            self.map.remove_layer(layer)
        
        # Set the new basemap
        self.map.basemap = new_basemap
        
        with self.output:
            print(f"🗺️ Switched to {new_basemap_name} basemap")
    
    def _handle_draw(self, target, action, geo_json: Dict[str, Any]) -> None:
        """
        Handle drawing events.
        
        Args:
            target: The draw control target
            action: The action performed
            geo_json (Dict): GeoJSON representation of the drawn feature
        """
        if geo_json['geometry']['type'] in ['Polygon', 'Rectangle']:
            self._add_polygon(geo_json)
            self._update_wkt_output()
            
            with self.output:
                print(f"✅ {geo_json['geometry']['type']} drawn successfully!")
    
    def _handle_data_change(self, change) -> None:
        """
        Handle changes in the draw control data (edits and deletions).
        
        Args:
            change: The change event containing new and old data
        """
        try:
            # Update our polygon list based on current draw control data
            current_data = change['new']
            
            # Clear current polygons and rebuild from draw control data
            self.polygons.clear()
            
            # Handle different data structures
            if isinstance(current_data, dict) and 'features' in current_data:
                # GeoJSON FeatureCollection format
                features = current_data['features']
            elif isinstance(current_data, list):
                # Direct list of features
                features = current_data
            else:
                # Unknown format, skip
                return
            
            # Process each feature
            for feature in features:
                if isinstance(feature, dict) and 'geometry' in feature:
                    if feature['geometry']['type'] in ['Polygon', 'Rectangle']:
                        self._add_polygon(feature)
            
            self._update_wkt_output()
            
            with self.output:
                if len(self.polygons) == 0:
                    print("🗑️ All polygons cleared!")
                else:
                    print(f"✏️ Polygons updated! Current count: {len(self.polygons)}")
                    
        except Exception as e:
            with self.output:
                print(f"⚠️ Error handling data change: {str(e)}")
    
    def _add_polygon(self, geo_json: Dict[str, Any]) -> None:
        """
        Add a polygon to the internal storage.
        
        Args:
            geo_json (Dict): GeoJSON representation of the polygon
        """
        polygon_data = {
            'id': len(self.polygons),
            'geo_json': geo_json,
            'coordinates': geo_json['geometry']['coordinates']
        }
        self.polygons.append(polygon_data)
    
    def _coordinates_to_wkt(self, coordinates: List[List[List[float]]]) -> str:
        """
        Convert polygon coordinates to WKT format.
        
        Args:
            coordinates (List): Polygon coordinates in GeoJSON format
            
        Returns:
            str: WKT representation of the polygon
        """
        # Handle both Polygon and Rectangle geometries
        if len(coordinates) > 0 and len(coordinates[0]) > 0:
            # Get the exterior ring coordinates
            exterior_coords = coordinates[0]
            
            # Ensure the polygon is closed (first and last points are the same)
            if exterior_coords[0] != exterior_coords[-1]:
                exterior_coords.append(exterior_coords[0])
            
            # Convert to WKT format (lon lat)
            coord_strings = [f"{lon} {lat}" for lon, lat in exterior_coords]
            return f"POLYGON(({', '.join(coord_strings)}))"
        
        return ""
    
    def _update_wkt_output(self) -> None:
        """
        Update the WKT output textarea with current polygons.
        """
        if not self.polygons:
            self.wkt_output.value = ""
            return
        
        wkt_strings = []
        for i, polygon in enumerate(self.polygons):
            wkt = self._coordinates_to_wkt(polygon['coordinates'])
            if wkt:
                wkt_strings.append(f"-- Polygon {i+1} --\n{wkt}")
        
        self.wkt_output.value = "\n\n".join(wkt_strings)
    
    def _clear_all(self, button) -> None:
        """
        Clear all drawn polygons.
        
        Args:
            button: The button widget that triggered this event
        """
        self.draw_control.clear()
        self.polygons.clear()
        self.wkt_output.value = ""
        
        with self.output:
            print("🧹 All polygons cleared!")
    
    def _copy_wkt(self, button) -> None:
        """
        Copy WKT to clipboard (display instruction).
        
        Args:
            button: The button widget that triggered this event
        """
        with self.output:
            print("📋 Select and copy the WKT text from the output area above.")
    
    def _load_wkt(self, button) -> None:
        """
        Load WKT string and display on map.
        
        Args:
            button: The button widget that triggered this event
        """
        wkt_string = self.wkt_input.value.strip()
        if not wkt_string:
            with self.output:
                print("❌ Please enter a WKT string to load.")
            return
        
        try:
            # Parse WKT string
            geometry = wkt.loads(wkt_string)
            
            if geometry.geom_type != 'Polygon':
                with self.output:
                    print(f"❌ Only Polygon geometries are supported. Got: {geometry.geom_type}")
                return
            
            # Convert to GeoJSON and add to map
            geo_json_feature = {
                'type': 'Feature',
                'geometry': mapping(geometry),
                'properties': {}
            }
            
            # Add as GeoJSON layer
            geojson_layer = GeoJSON(
                data=geo_json_feature,
                name=f"Loaded Polygon",
                style={
                    'fillColor': '#ffaa00',
                    'color': '#ff8800',
                    'fillOpacity': 0.3,
                    'weight': 2
                }
            )
            self.map.add_layer(geojson_layer)
            
            # Fit map to geometry bounds
            bounds = geometry.bounds  # (minx, miny, maxx, maxy)
            self.map.fit_bounds([[bounds[1], bounds[0]], [bounds[3], bounds[2]]])
            
            with self.output:
                print(f"✅ WKT polygon loaded successfully!")
                
        except Exception as e:
            with self.output:
                print(f"❌ Error loading WKT: {str(e)}")
    
    def get_wkt_polygons(self) -> List[str]:
        """
        Get all drawn polygons as WKT strings.
        
        Returns:
            List[str]: List of WKT strings for all polygons
        """
        wkt_polygons = []
        for polygon in self.polygons:
            wkt = self._coordinates_to_wkt(polygon['coordinates'])
            if wkt:
                wkt_polygons.append(wkt)
        return wkt_polygons
    
    def display(self) -> VBox:
        """
        Display the complete polygon tool interface.
        
        Returns:
            VBox: The complete UI widget
        """
        # Instructions
        instructions = HTML(
            value="""
            <div style="background-color: #f0f8ff; padding: 10px; border-radius: 5px; margin-bottom: 10px;">
                <h4>🗺️ How to use the Interactive Polygon Tool:</h4>
                <ul>
                    <li><strong>Draw:</strong> Use the polygon or rectangle tools on the map</li>
                    <li><strong>Edit:</strong> Click on a drawn polygon and drag the vertices</li>
                    <li><strong>Delete:</strong> Select a polygon and use the delete tool</li>
                    <li><strong>Export:</strong> Copy the WKT coordinates for use in searches</li>
                    <li><strong>Import:</strong> Load existing WKT strings to visualize</li>
                    <li><strong>Switch Basemaps:</strong> Use the dropdown to change background imagery</li>
                </ul>
                <h4>🗺️ Available Basemap Categories:</h4>
                <ul>
                    <li><strong>🛰️ Satellite & Imagery:</strong> High-resolution satellite views, NASA imagery</li>
                    <li><strong>🗺️ Street & Road Maps:</strong> Traditional street maps and navigation-friendly views</li>
                    <li><strong>⛰️ Topographic & Terrain:</strong> Elevation, terrain, and physical geography maps</li>
                    <li><strong>🌑 Dark Themes:</strong> Dark-styled maps for reduced eye strain</li>
                    <li><strong>🎨 Artistic & Special:</strong> Watercolor, artistic, and specialized map styles</li>
                    <li><strong>❄️ Specialized:</strong> Arctic, climate, and scientific data visualizations</li>
                </ul>
            </div>
            """
        )
        
        # Button row
        button_row = HBox([self.clear_button, self.copy_button])
        
        # Basemap controls
        controls_section = VBox([])
        if self.show_basemap_switcher:
            controls_section.children = [
                HTML("<h4>🗺️ Map Controls</h4>"),
                self.basemap_dropdown
            ]
        
        # Load WKT section
        load_section = VBox([
            HTML("<h4>📥 Load WKT Polygon</h4>"),
            self.wkt_input,
            self.load_button
        ])
        
        # Output section
        output_section = VBox([
            HTML("<h4>📤 WKT Output</h4>"),
            self.wkt_output,
            button_row
        ])
        
        # Build the layout conditionally
        layout_children = [instructions, self.map]
        
        if self.show_basemap_switcher:
            layout_children.append(controls_section)
        
        layout_children.extend([
            load_section,
            output_section,
            HTML("<h4>📋 Messages</h4>"),
            self.output
        ])
        
        return VBox(layout_children)

print("✅ InteractivePolygonTool class created successfully!")

✅ InteractivePolygonTool class created successfully!


## Create and Display the Interactive Tool

Now let's create an instance of our polygon tool and display it:

In [6]:
# Create the interactive polygon tool
# You can customize the initial location and zoom level
polygon_tool = InteractivePolygonTool(
    center=(45.0, 0.0),  # Initial center (latitude, longitude)
    zoom=3,  # Initial zoom level
    basemap=basemaps.OpenStreetMap.Mapnik  # You can change the basemap
)

# Display the tool
display(polygon_tool.display())

## Using the Tool with phidown

Once you've drawn polygons, you can use them directly with the phidown library for satellite data searches:

In [8]:
# Example of how to use with phidown (uncomment to use)
import sys
from phidown.search import CopernicusDataSearcher

wkt_polygons = polygon_tool.get_wkt_polygons()
    
if not wkt_polygons:
    print("❌ No polygons drawn yet. Please draw a polygon on the map first.")
    sys.exit(1)

print(f"📍 Found {len(wkt_polygons)} polygon(s):")

for i, wkt_polygon in enumerate(wkt_polygons):
    print(f"\nPolygon {i+1}:")
    print(f"WKT: {wkt_polygon}")


    searcher = CopernicusDataSearcher()
    searcher._query_by_filter(
        collection_name='SENTINEL-2',
        product_type="S2MSI1C",
        cloud_cover_threshold=0.5,
        aoi_wkt=wkt_polygon,  # Use the drawn polygon as AOI
        start_date='2022-06-01T00:00:00',
        end_date='2023-06-03T00:00:00',
        top=10
    )
    df = searcher.execute_query()
    print(f"Found {len(df)} products for this polygon")
    # save the results to a CSV file
    df.to_csv(f"sentinel2_products_polygon_{i+1}.csv", index=False)
    print(f"Results saved to sentinel2_products_polygon_{i+1}.csv")


📍 Found 1 polygon(s):

Polygon 1:
WKT: POLYGON((14.573107 40.734371, 14.590356 40.722012, 14.619792 40.721882, 14.621508 40.739573, 14.590356 40.746271, 14.573107 40.734371))
Found 10 products for this polygon
Results saved to sentinel2_products_polygon_1.csv


## Advanced Features

### Custom Styled Map with Satellite Imagery

In [5]:
# Create a tool with satellite imagery basemap
satellite_tool = InteractivePolygonTool(
    center=(37.7749, -122.4194),  # San Francisco coordinates
    zoom=5,
    basemap=basemaps.Esri.WorldImagery)
display(satellite_tool.display())

## 🗺️ Expanded Basemap Collection

Your polygon tool now includes **30+ high-quality basemap layers** organized into 6 categories:

### 🛰️ Satellite & Imagery Basemaps
- **Esri World Imagery**: High-resolution satellite imagery
- **NASA Blue Marble**: Earth from space view
- **NASA Earth at Night**: Night lights and urban areas
- **NASA MODIS/VIIRS**: Real satellite data with true colors
- **Stadia Satellite**: Alternative satellite imagery

### ⛰️ Topographic & Terrain Basemaps
- **Esri World Topo**: Detailed topographic maps
- **Esri World Terrain/Physical**: Terrain and elevation data
- **OpenTopoMap**: Open-source topographic mapping
- **Stadia Terrain/Outdoors**: Hiking and outdoor activities

### 🌑 Dark Theme Basemaps
- **CartoDB Dark Matter**: Clean dark theme
- **Stadia Alidade Dark**: Modern dark styling
- **Stadia Toner**: High-contrast black and white

### 🎨 Artistic & Special Basemaps
- **Stadia Watercolor**: Beautiful artistic watercolor style
- **Esri National Geographic**: Classic National Geographic styling
- **Esri Ocean Basemap**: Optimized for marine areas

### ❄️ Specialized Scientific Basemaps
- **Arctic Imagery**: Polar region visualization
- **NASA Ice Velocity**: Ice movement data
- **NASA Land Surface Temperature**: Climate data visualization
- **NASA Snow Cover**: Snow and ice coverage

### How to Switch Basemaps
1. Use the **Basemap dropdown** in the tool interface
2. Basemaps are organized with emoji prefixes for easy identification
3. Each category serves different analysis purposes

---

In [ ]:
# Examples of creating tools with different specialized basemaps

# 1. Satellite Imagery Tool (Perfect for visual identification)
satellite_tool = InteractivePolygonTool(
    center=(37.7749, -122.4194),  # San Francisco
    zoom=12,
    basemap=basemaps.Esri.WorldImagery
)

# 2. NASA Night Lights Tool (Great for urban area analysis)
night_tool = InteractivePolygonTool(
    center=(40.7589, -73.9851),  # New York City
    zoom=10,
    basemap=basemaps.NASAGIBS.ViirsEarthAtNight2012
)

# 3. Topographic Tool (Perfect for terrain analysis)
topo_tool = InteractivePolygonTool(
    center=(46.8566, 8.6476),  # Swiss Alps
    zoom=10,
    basemap=basemaps.Esri.WorldTerrain
)

# 4. Arctic Research Tool (For polar studies)
arctic_tool = InteractivePolygonTool(
    center=(90.0, 0.0),  # North Pole
    zoom=4,
    basemap=basemaps.Esri.ArcticImagery
)

# 5. Dark Theme Tool (Reduced eye strain)
dark_tool = InteractivePolygonTool(
    center=(51.5074, -0.1278),  # London
    zoom=10,
    basemap=basemaps.CartoDB.DarkMatter
)

print("🎯 Specialized basemap tools created!")
print("\n🛰️ Available specialized tools:")
print("  • satellite_tool - High-res satellite imagery")
print("  • night_tool - NASA Earth at night")
print("  • topo_tool - Topographic/terrain mapping")
print("  • arctic_tool - Arctic region analysis")
print("  • dark_tool - Dark theme interface")
print("\n💡 Use display(tool_name.display()) to show any tool")
print("💡 Each tool has access to 30+ basemaps via the dropdown!")

# Display the satellite tool as an example
print("\n🖼️ Displaying satellite imagery tool:")
display(satellite_tool.display())

## 📊 Basemap Categories & Use Cases

### 🛰️ **Satellite & Imagery** - Best for:
- **Visual identification** of geographic features
- **Land use classification** and change detection
- **Coastal monitoring** and environmental studies
- **Urban planning** and infrastructure analysis

**Recommended basemaps:**
- `🛰️ Esri World Imagery` - Highest quality, most recent
- `🌍 NASA Blue Marble` - Global perspective, cloud-free
- `🌙 NASA Earth at Night` - Urban areas, light pollution

### ⛰️ **Topographic & Terrain** - Best for:
- **Elevation analysis** and watershed studies
- **Hiking and outdoor activity** planning
- **Geological surveys** and slope analysis
- **Natural disaster** risk assessment

**Recommended basemaps:**
- `⛰️ Esri World Topo` - Comprehensive topographic data
- `⛰️ OpenTopoMap` - Open-source alternative
- `⛰️ Esri World Physical` - Physical geography focus

### 🌑 **Dark Themes** - Best for:
- **Night work** and reduced eye strain
- **Data visualization** with bright overlays
- **Modern interface** aesthetics
- **Focus on drawn polygons** rather than basemap

**Recommended basemaps:**
- `🌑 CartoDB Dark Matter` - Clean, minimal dark theme
- `🌑 Stadia Alidade Dark` - Modern dark styling

### ❄️ **Specialized Scientific** - Best for:
- **Climate research** and polar studies
- **Environmental monitoring** (temperature, snow cover)
- **Oceanographic** and Arctic research
- **Scientific data visualization**

**Recommended basemaps:**
- `❄️ Esri Arctic Imagery` - Polar region focus
- `🌡️ NASA Land Surface Temp` - Climate analysis
- `☃️ NASA Snow Cover` - Seasonal studies

---

### 📝 Quick Reference: When to Use Each Category

| **Analysis Type** | **Recommended Category** | **Best Basemap** |
|-------------------|-------------------------|------------------|
| Satellite data search | 🛰️ Satellite & Imagery | Esri World Imagery |
| Terrain analysis | ⛰️ Topographic | Esri World Topo |
| Urban studies | 🜙 Night Lights | NASA Earth at Night |
| Arctic research | ❄️ Specialized | Arctic Imagery |
| Long work sessions | 🌑 Dark Themes | CartoDB Dark Matter |
| Ocean/Marine areas | 🌊 Special | Esri Ocean Basemap |

### Example: Load a Predefined Polygon

Let's demonstrate loading a WKT polygon (example from Sicily, Italy):

In [ ]:
# Example WKT polygon for Sicily area (from the original notebook)
sicily_wkt = "POLYGON((14.889908 37.722392, 14.960632 37.672408, 15.113068 37.695231, 15.109634 37.804359, 14.956512 37.827684, 14.889908 37.722392))"

# Create a tool centered on Sicily
sicily_tool = InteractivePolygonTool(
    center=(37.75, 14.98),  # Sicily coordinates
    zoom=10,
    basemap=basemaps.OpenStreetMap.Mapnik
)

print(f"🇮🇹 Sicily polygon tool created!")
print(f"Example WKT for Sicily: {sicily_wkt}")
print("\nTo load this polygon:")
print("1. Display the tool using: display(sicily_tool.display())")
print("2. Paste the WKT string in the 'Load WKT' text area")
print("3. Click 'Load WKT' button")

# Uncomment to display:
# display(sicily_tool.display())

## Integration Example with phidown

Here's a complete example showing how to integrate the polygon tool with a phidown search:

In [ ]:
def search_with_drawn_polygon():
    """
    Complete example of using drawn polygons for satellite data search.
    """
    from phidown.search import CopernicusDataSearcher
    import pandas as pd
    
    # Get WKT polygons from the tool
    wkt_polygons = polygon_tool.get_wkt_polygons()
    
    if not wkt_polygons:
        print("❌ Please draw a polygon first using the tool above.")
        return None
    
    # Use the first polygon for search
    aoi_wkt = wkt_polygons[0]
    print(f"🔍 Searching with polygon: {aoi_wkt[:100]}...")
    
    try:
        # Configure search
        searcher = CopernicusDataSearcher()
        searcher._query_by_filter(
            collection_name='SENTINEL-2',
            product_type="S2MSI1C",
            orbit_direction=None,
            cloud_cover_threshold=20,  # Max 20% cloud cover
            aoi_wkt=aoi_wkt,  # Use the drawn polygon
            start_date='2024-05-01T00:00:00',
            end_date='2024-05-31T00:00:00',
            top=5  # Limit to 5 results for demo
        )
        
        # Execute search
        df = searcher.execute_query()
        
        if len(df) > 0:
            print(f"✅ Found {len(df)} Sentinel-2 products!")
            searcher.display_results(top_n=3)
            return df
        else:
            print("❌ No products found for the specified criteria.")
            return None
            
    except Exception as e:
        print(f"❌ Error during search: {str(e)}")
        return None

print("🔧 Search function ready!")
print("After drawing a polygon, run: search_with_drawn_polygon()")

In [ ]:
search_with_drawn_polygon()